In [1]:
from pyrosm import OSM
from pyrosm import get_data
import geopandas as gpd

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
osmdata_transfer='java-latest-internal.osm.pbf'

In [3]:
from google.cloud import storage

bucket_name = 'bps-gcp-bucket'
source_blob_name = 'Degree of Urbanisation/OSM/'+osmdata_transfer
destination_file_name ='data/input/OSM/'+osmdata_transfer
#DOWNLOAD
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(source_blob_name)
blob.download_to_filename(destination_file_name)
print('Blob {} downloaded to {}.'.format(source_blob_name, destination_file_name))

Blob Degree of Urbanisation/OSM/java-latest-internal.osm.pbf downloaded to data/input/OSM/java-latest-internal.osm.pbf.


In [ ]:
osm = OSM('data/input/OSM/'+osmdata_transfer)
drive_net = osm.get_network(network_type="driving")

The reference document used is "Rural Access Index (RAI) Supplemental Guidelines (ReCAP, 2019)"

Here, we'll filter network data which only associated with all-seasoned road. Using the OSM data, we can get the all-seasoned road based on two variable: "surface" and "highway"

Based on this link: https://wiki.openstreetmap.org/wiki/Key:highway ; the preferable highway data is only: ['secondary', 'tertiary', 'primary','trunk','motorway' ]

For the surface data, we have to check manually! Based on this link: https://wiki.openstreetmap.org/wiki/Key:surface ; we will select "PAVED" road only. But, sometimes, the volunteer, put Indonesian names, therefore we have to do manual checking

In [ ]:
drive_net.surface.unique()

In [ ]:
drive_net.highway.unique()

In [ ]:
highway_l=['secondary', 'tertiary', 'primary',
       'trunk','motorway' ]

In [10]:
surface_l=['paved', 'asphalt', 'cobblestone', None,
        'paving_stones', 'gravel;asphalt', 'concrete',
        'asphalt_ruts', 'Hotmix', 'dirt;asphalt', 'metal',
        'sett', 'concrete:plates', 'Aspal dan Tanah',
        'permukaan tanah dan Beton', 'Tanah dan Aspal',
        'Aspal dan Beton', 'Tanah dan Beton', 'concrete:lanes',
       'wood', 'Trotoar', 'PAVING BLOK', 'Hotmix dan Tanah',
       'paving_stones:30', 'driveway', 'unhewn_cobblestone']

In [11]:
gf=gpd.read_file('data/input/Master_Data/Prov_20201_ByIndo(1).shp')

In [12]:
highway_=drive_net.query('highway in @highway_l').query('surface in @surface_l')

In [13]:
data_highway=gpd.sjoin(highway_,gf,how='inner',op='intersects')

/opt/tljh/user/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [14]:
data_h=data_highway[['kdprov','geometry']].dissolve(by='kdprov').reset_index()

In [15]:
for i in data_h.kdprov:
    data_h.query('kdprov==@i').to_file('data/input/OSM/'+i+'_allseasonedroad_OSM.gpkg',driver='GPKG')